# Important: Read this!

This recipe does not work with the standard conda environment.

If you are in the standard environment, do this:

1. Stop Jupyter
2. Activate QIIME2 environment on conda
3. Do `jupyter serverextension enable --py qiime2 --sys-prefix`
4. Start Jupyter inside QIIME2 environment

Note that other recipes will not work inside this environment. 

# Getting the data

In [1]:
!rm -rf data
!mkdir data

!wget -O "sample-metadata.tsv" "https://data.qiime2.org/2018.6/tutorials/moving-pictures/sample_metadata.tsv"

!wget -O "data/barcodes.fastq.gz" "https://data.qiime2.org/2018.6/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz"

!wget -O "data/sequences.fastq.gz" "https://data.qiime2.org/2018.6/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz"

--2018-09-07 21:32:17--  https://data.qiime2.org/2018.6/tutorials/moving-pictures/sample_metadata.tsv
Resolving data.qiime2.org... 104.27.171.158, 104.27.170.158
Connecting to data.qiime2.org|104.27.171.158|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://docs.google.com/spreadsheets/d/14q7ScySUpJqUDHbbSLkPXSTHd337dYuaQxWMsZJzSVo/export?gid=0&format=tsv [following]
--2018-09-07 21:32:18--  https://docs.google.com/spreadsheets/d/14q7ScySUpJqUDHbbSLkPXSTHd337dYuaQxWMsZJzSVo/export?gid=0&format=tsv
Resolving docs.google.com... 64.233.185.113, 64.233.185.100, 64.233.185.139, ...
Connecting to docs.google.com|64.233.185.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tab-separated-values]
Saving to: ‘sample-metadata.tsv’

sample-metadata.tsv     [ <=>                ]   3.70K  --.-KB/s    in 0.02s   

2018-09-07 21:32:19 (220 KB/s) - ‘sample-metadata.tsv’ saved [3785]

--2018-09-07 21:32:19--  https://data

# The recipe

In [2]:
import pandas as pd

from qiime2.metadata.metadata import Metadata
from qiime2.metadata.metadata import CategoricalMetadataColumn
from qiime2.sdk import Artifact
from qiime2.sdk import PluginManager
from qiime2.sdk import Result

In [3]:
#After convert
pm = PluginManager()
#print(pm.plugins)
demux_plugin = pm.plugins['demux']
demux_emp_single = demux_plugin.actions['emp_single']
demux_summarize = demux_plugin.actions['summarize']

In [4]:
print(demux_emp_single.description)
emp_single_signature = demux_emp_single.signature
print(emp_single_signature.inputs)
print(emp_single_signature.parameters)
print(emp_single_signature.outputs)
seq_input = emp_single_signature.inputs['seqs']

Demultiplex sequence data (i.e., map barcode reads to sample ids) for data generated with the Earth Microbiome Project (EMP) amplicon sequencing protocol. Details about this protocol can be found at http://www.earthmicrobiome.org/protocols-and-standards/
OrderedDict([('seqs', ParameterSpec(qiime_type=EMPPairedEndSequences | EMPSingleEndSequences | RawSequences, view_type=<class 'q2_demux._demux.BarcodeSequenceFastqIterator'>, default=NOVALUE, description='The single-end sequences to be demultiplexed.'))])
OrderedDict([('barcodes', ParameterSpec(qiime_type=MetadataColumn[Categorical], view_type=<class 'qiime2.metadata.metadata.CategoricalMetadataColumn'>, default=NOVALUE, description='The sample metadata column containing the per-sample barcodes.')), ('rev_comp_barcodes', ParameterSpec(qiime_type=Bool, view_type=<class 'bool'>, default=False, description='If provided, the barcode sequence reads will be reverse complemented prior to demultiplexing.')), ('rev_comp_mapping_barcodes', Param

In [5]:
#convert
data_type = 'EMPSingleEndSequences'
conv = Artifact.import_data(data_type, 'data')
conv.save('out.qza')

'out.qza'

In [6]:
#demux
seqs = Result.load('out.qza')
metadata = Metadata.load('sample-metadata.tsv')
metadata_df = metadata.to_dataframe()
demux = demux_emp_single(seqs, CategoricalMetadataColumn(metadata_df['BarcodeSequence']))

demux.per_sample_sequences.save('demux.qza')

'demux.qza'

In [7]:
sum_data = demux_summarize(demux.per_sample_sequences)

print(dir(sum_data))
print(type(sum_data.visualization))
print(dir(sum_data.visualization))
sum_data.visualization

['__add__', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_fields', 'count', 'index', 'visualization']
<class 'qiime2.sdk.result.Visualization'>
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_alias', '_archiver', '_destructor', '_from_data_dir', '_is_valid_type', '_repr_html_', 'citations', 'export_data', 'extension', 'extract', 'format', 'get_index_paths', 'lo

<visualization: Visualization uuid: 1ae76e7b-a893-40d8-a886-be6d62f7adc5>

In [8]:
#Quality control
dada2_plugin = pm.plugins['dada2']
dada2_denoise_single = dada2_plugin.actions['denoise_single']
qual_control = dada2_denoise_single(demultiplexed_seqs=demux.per_sample_sequences,
                                    trunc_len=120, trim_left=0)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_single.R /tmp/qiime2-archive-3c4tvb4c/e6c52b18-b637-4bb2-a8b1-7e99cb0b794a/data /tmp/tmpovwabzt8/output.tsv.biom /tmp/tmpovwabzt8/track.tsv /tmp/tmpovwabzt8 120 0 2.0 2 Inf consensus 1.0 1 1000000 NULL 16



In [9]:
qual_control.denoising_stats.save('stats-dada2.qza')
qual_control.representative_sequences.save('rep-seqs.qza')
qual_control.table.save('table.qza')

'table.qza'

In [10]:
metadata_plugin = pm.plugins['metadata']
metadata_tabulate = metadata_plugin.actions['tabulate']
stats_meta = metadata_tabulate(input=qual_control.denoising_stats.view(Metadata))
stats_meta.visualization

<visualization: Visualization uuid: 3056b50f-32de-474e-bc9e-8c9dbb3db4d7>

In [11]:
# phylogenetic tree
phylogeny_plugin = pm.plugins['phylogeny']
phylogeny_align_mafft = phylogeny_plugin.actions['align_to_tree_mafft_fasttree']
tree = phylogeny_align_mafft(sequences=qual_control.representative_sequences)

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 1 /tmp/qiime2-archive-tpmkm6pw/0ee77895-3c42-4870-b5ef-e40dfcbb45e9/data/dna-sequences.fasta

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /tmp/qiime2-archive-ly0ttq0q/d4108d7d-7b18-40c2-9b27-5d22804d345e/data/aligned-dna-sequences.fasta



In [12]:
tree.tree.export_data(output_dir='.')

In [13]:
#Talk about zips

In [14]:
# diversity analysis
diversity_plugin = pm.plugins['diversity']
diversity_core_metrics = diversity_plugin.actions['core_metrics_phylogenetic']
core_metrics = diversity_core_metrics(phylogeny=tree.rooted_tree,
                                      table=qual_control.table, sampling_depth=1109, metadata=metadata)

/home/tiago_antao/anaconda3/envs/qiime2-2018.8/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/tiago_antao/anaconda3/envs/qiime2-2018.8/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.0120894730791837 and the largest is 3.6251052559305768.
  RuntimeWarning
/home/tiago_antao/anaconda3/envs/qiime2-2018.8/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negativ

In [15]:
core_metrics.bray_curtis_emperor

<visualization: Visualization uuid: 4dfb4238-63df-47b8-afde-4cea32c5752e>